### Season 4 Phenotype Table
#### Started November 2019 after GenoPhenoEnvo Kickoff
#### Columns
* `range`
* `column`
* `canopy_height`
* `max_canopy_height`
* `days_to_` multiple values

In [ ]:
# %pwd
%cd '/Users/ejcain/Dev/UA-AG/phenotypes/terraref-datasets/'

In [ ]:
import numpy as np
import pandas as pd

* Season 4 data queried from betydb using R, then downloaded as csv
* Some columns have already been dropped in the very messy season_4_transformation notebook

In [ ]:
df = pd.read_csv('data/interim/more_traits_after_feedback.csv')
print(df.shape)
df.head()

In [ ]:
# df.trait.astype('object').combine(df['mean'], func=)    

#### New Trait Columns

Change column name `mean` due to being a keyword and also confusing

In [ ]:
df = df.rename({'mean': 'value'}, axis=1)
df.tail()

In [ ]:
df.shape

Drop `trait_description` and `units`

In [ ]:
df.drop(labels='trait_description', axis=1, inplace=True)
df.drop(labels='units', axis=1, inplace=True)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

* Set sitename as index
* 
*

In [ ]:
df = df.set_index(keys='sitename')
print(df.shape)
df.head()

In [ ]:
df.dtypes

#### Add `column` and `range` columns for easier location identification

In [ ]:
df.head()

In [ ]:
sitename_values = df.index.values
print(len(df))
print(df.index[:10])

In [ ]:
no_e_w_names = []

for name in sitename_values:
    
    if name.endswith(' W') | name.endswith(' E'):
        name = name[:-2]
        no_e_w_names.append(name)
        
    else:
        no_e_w_names.append(name)

print(len(no_e_w_names))
print(no_e_w_names[:10])

In [ ]:
# this cell should have no output

for name in no_e_w_names:
    
    if name.endswith(' W'):
        print(name)
        
    if name.endswith(' E'):
        print(name)
        
    if name.endswith(' '):
        print('This is wrong too.')

In [ ]:
test_sitename_string = no_e_w_names[42]
test_sitename_string

In [ ]:
split_string = test_sitename_string.split(' ')

In [ ]:
print(split_string[6])
print(split_string[8])

In [ ]:
ranges = []
columns = []

for name in no_e_w_names:
    
    split_name = name.split(' ')
    
    range_value = split_name[6]
    ranges.append(range_value)
    
    column_value = split_name[8]
    columns.append(column_value)
    
print(f'There are {len(ranges)} range values!')
print(f'First three range values: {ranges[:3]}')

print(f'There are {len(columns)} column values!')
print(f'Here are the first three column values! {columns[:3]}')

# print(f'All of these numbers should be {len(sitename_index)}')

In [ ]:
df.head(3)

In [ ]:
df['range_location'] = ranges
df['column_location'] = columns

df.tail()

In [ ]:
# check that `E` and `W` columns have accurate range and column values

df.loc[df.index.str.endswith(' E')].sample(n=7)

In [ ]:
# check W's

df.loc[df.index.str.endswith(' W')].sample(n=4)

In [ ]:
# sitename_index.to_csv('column_and_range_df.csv')

In [ ]:
df.shape

#### Create empty DataFrame with trait columns to merge with df

In [ ]:
trait_columns = ['proximal_air_temperature', 'leaf_temperature_differential', 'surface_temperature',
                'aboveground_dry_biomass', 'seedling_emergence_rate', 'canopy_height', 
                'flag_leaf_emergence_rate', 'flowering_time', 'grain_stage_time', 'canopy_cover',
                'emergence_count']

empty_df = pd.DataFrame(index=df.index, columns=trait_columns)

In [ ]:
empty_df.head()

In [ ]:
empty_df.shape

In [ ]:
df.head(2)

In [ ]:
# empty_df.to_csv('empty_df_with_index.csv')
# df.to_csv('df_to_merge_with_df.csv')

In [ ]:
print(empty_df.shape)
print(df.shape)

In [ ]:
# check that the indices are identical

print(pd.Series(empty_df.index.isin(df.index)).value_counts())

In [ ]:
what_is_happening = pd.concat([df, empty_df.reindex(df.index)], axis=1)
what_is_happening.shape

In [ ]:
# the key for a successful merge / join / concat is to use an improper name for the resulting df

In [ ]:
what_is_happening.head()

In [ ]:
what_is_happening.columns

In [ ]:
new_df = what_is_happening.copy()

In [ ]:
new_df.to_csv('merged_df_0.csv')

#### _Now_ populate trait columns

In [ ]:
#trait_columns = ['proximal_air_temperature', 'leaf_temperature_differential', 'surface_temperature',
#                'aboveground_dry_biomass', 'seedling_emergence_rate', 'canopy_height', 
#                'flag_leaf_emergence_rate', 'flowering_time', 'grain_stage_time', 'canopy_cover',
#                'emergence_count']

#for trait in trait_columns:

    #for index, row in new_df.iterrows():
        #if row['trait'] == trait:
            #new_df.loc[index, [trait]] = row['value']
        
#new_df.head()

In [ ]:
trait_columns = ['proximal_air_temperature', 'leaf_temperature_differential', 'surface_temperature',
                'aboveground_dry_biomass', 'seedling_emergence_rate', 'canopy_height', 
                'flag_leaf_emergence_rate', 'flowering_time', 'grain_stage_time', 'canopy_cover',
                'emergence_count']


counter = 0

for index, row in df.iterrows():
    if counter % 1000 == 0:
        print(counter)
    counter = counter + 1
    for trait in trait_columns:
        if row['trait'] == trait:
            new_df.loc[index, [trait]] = row['value']
        
new_df.head()

In [ ]:
new_df.to_csv('lots_of_traits_df.csv')

******

In [ ]:
df = pd.read_csv('data/interim/lots_of_traits_df_0.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

Are all these value counts accounted for?!

In [ ]:
df.trait.value_counts()

Correct `flag_leaf_emergence_time` column

In [ ]:
df = df.assign(flag_leaf_emergence_time = np.nan)
df.tail()

In [ ]:
counter = 0

for index, row in df.iterrows():
        if counter % 1000 == 0:
            print(counter)
        counter = counter + 1
        if row['trait'] == 'flag_leaf_emergence_time':
            df.loc[index, ['flag_leaf_emergence_time']] = row['value']

In [ ]:
df.drop(labels='flag_leaf_emergence_rate', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
sitenames = df.sitename.values
print(sitenames[0])
print(len(sitenames))

In [ ]:
this_is_an_empty_string = ' '
string_names = []
for name in sitenames:
    
    new_name_list = name.split(' ')[:9]
    # print(new_name_list)
    new_name = this_is_an_empty_string.join(new_name_list)
    # print(new_name)
    string_names.append(new_name)

string_names[:10]

In [ ]:
len(string_names)

In [ ]:
df['string_names_no_e_w'] = string_names
df.head()

In [ ]:
# fill NaNs with empty string

df.fillna(value=' ', axis=1, inplace=True)

In [ ]:
print(df.isnull().sum())
df.head()

In [ ]:
df.to_csv('lots_of_traits_df_1.csv')

#### Drop Duplicates?
With separate trait columns, some rows could be dropped if they have duplicate:
* `raw_date`
* `sitename`
* `site_id`
* `cultivar_id`

In [ ]:
df.duplicated(subset=['raw_date', 'sitename', 'site_id', 'cultivar_id'], keep=False).value_counts()

In [ ]:
df.duplicated(subset=['raw_date', 'sitename', 'site_id', 'cultivar_id', 'trait'], keep=False).value_counts()

In [ ]:
duplicate_df = df[df.duplicated(subset=['raw_date', 'sitename', 'site_id', 'cultivar_id', 'trait'], keep=False)]
duplicate_df.shape

In [ ]:
duplicate_df.head()

In [ ]:
# should have zero output

for name in string_names:
    
    if name.endswith(' W'):
        print(name)
    
    elif name.endswith(' E'):
        print(name)

In [ ]:
df.columns

#### Take averages for E and W sites

In [ ]:
e_w_sites = df.loc[(df.sitename.str.endswith(' E')) | (df.sitename.str.endswith(' W'))]
print(e_w_sites.shape)
e_w_sites.head()

Find matching plots

In [ ]:
test_name = df.iloc[0].sitename.split(' ')[:9]
test_name

In [ ]:
this_is_an_empty_string = ' '
this_is_an_empty_string.join(test_name)

In [ ]:
sitenames = e_w_sites.sitename.values
print(sitenames[0])
print(len(sitenames))

In [ ]:
this_is_an_empty_string = ' '
string_names = []
for name in sitenames:
    
    new_name_list = name.split(' ')[:9]
    # print(new_name_list)
    new_name = this_is_an_empty_string.join(new_name_list)
    # print(new_name)
    string_names.append(new_name)
    
len(string_names)
string_names[:10]

In [ ]:
e_w_sites['new_sitenames'] = string_names
e_w_sites.head()

In [ ]:
len(e_w_sites[e_w_sites.duplicated(['new_sitenames'], keep=False)])

In [ ]:
df.iloc[0].sitename

In [ ]:
for idx, row in e_w_sites.iterrows():
    
    print(idx)
    print(row.sitename.split(' ')[:9])
    break
    
    if row.sitename.split(' ')[:9]:
        pass

In [ ]:
# do not save index next time. . . 

df = pd.read_csv('lots_of_traits_df_1.csv')
df.head()

In [ ]:
sample_df = df.sample(n=10000)

In [ ]:
sample_df.dtypes

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df[['sitename', 'raw_date', 'canopy_height', 'string_names_no_e_w']].head()

In [ ]:
df.head()

In [ ]:
df["range_value"]  = df["string_names_no_e_w"].str.extract("Range (\d+)").astype(int)
df["column_value"] = df["string_names_no_e_w"].str.extract("Column (\d+)").astype(int)
df.head()

In [ ]:
# df['canopy_height'] = df['canopy_height'].astype(float)

In [ ]:
# for h in df['canopy_height']:
#     try:
#         this = float(h)
#     except:
#         print("here is h, h = {}".format(h))

In [ ]:
# df.groupby(by=["date", "range_value", "column_value", "cultivar"])['canopy_height'].agg(['mean'])

In [ ]:
df.columns

In [ ]:
df.flowering_time.unique()

In [ ]:
season_4_df = pd.read_csv('data/raw/mac_season_4.csv')
season_4_df.head()

In [ ]:
season_4_df.loc[season_4_df.trait == 'planter_seed_drop']

In [ ]:
season_4_df.trait.unique()

In [ ]:
season_4_df.columns

In [ ]:
season_4_df.shape

In [ ]:
season_4_df.raw_date.min()

In [ ]:
season_4_df.method_name.unique()

### Not Using This - Just Saving

In [ ]:
# Code from Rafa 

# from collections import defaultdict
# ​
# # make each trait a key, so you can make it a column later
# traits = df.trait.unique
# ​
# # create dictionary
# trait_columns_dict = {}
# ​
# # set all the traits as empty lists
# for trait in traits:
# 	trait_columns_dict[trait] = []
	
# # this is inefficient but it will work
# for trait, value in zip(df.trait, df.value):
# 	for col in traits:
# 		if trait == col:
# 			trait_columns_dict.append(value)
# 		else:
# 			trait_columns_dict.append(None)
# ​
# trait_columns_dict
# 	df